<a href="https://colab.research.google.com/github/zarrinan/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
# TODO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
from scipy.stats import chisquare

In [23]:
# Load the dataset, rename the columns, and see the head, description, A16 column value counts
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'
col_names = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']
df_raw = pd.read_csv(url, header=None, names=col_names)
print(df_raw.head())
print(df_raw.describe())
df_raw.A16.value_counts()

#I see from the data description file, the Number of Instances: 690, which is 
#confirmed by the count in dataframe description below. Also, the class distribution
#in dataset description is confirmed by the result of A16 column value_counts() result

  A1     A2     A3 A4 A5 A6 A7    A8 A9 A10  A11 A12 A13    A14  A15 A16
0  b  30.83  0.000  u  g  w  v  1.25  t   t    1   f   g  00202    0   +
1  a  58.67  4.460  u  g  q  h  3.04  t   t    6   f   g  00043  560   +
2  a  24.50  0.500  u  g  q  h  1.50  t   f    0   f   g  00280  824   +
3  b  27.83  1.540  u  g  w  v  3.75  t   t    5   t   g  00100    3   +
4  b  20.17  5.625  u  g  w  v  1.71  t   f    0   f   s  00120    0   +
               A3          A8        A11            A15
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


-    383
+    307
Name: A16, dtype: int64

In [0]:
#check if there are missing values, UCI says there are missing values:
#     A1:  12
#     A2:  12
#     A4:   6
#     A5:   6
#     A6:   9
#     A7:   9
#     A14: 13
      
df_raw.isnull().sum() # missing values are not entered as NaN
print(df_raw.A1.unique())
print(df_raw.A6.unique())
print(df_raw.A14.unique())

In [0]:
#replace missing values, denotes as '?' with NaN
df_raw.replace('?', np.nan, inplace = True)


In [28]:
#check for NaN in the dataframe
df_raw.isnull().sum()

#the data corresponds to the missing data information in the dataset

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [30]:
#before moving to data exploration, I'll drop nan values, since there are 690 instances of data
df_clean = df_raw.dropna()
df_clean.isnull().sum()

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
# TODO
# - divide dataframe into two by A16 class attribute

# - find 6 continious features = put the in a list named 'cont_feat'
# - run t_test for at least 3 continuous feature iterating through the 'cont_feat' list 
#   appliying t_test function, and saving the result in a dict 
# - select one feature that is extremely different between the classes, 
#   and another feature that is notably less different

# - find categorical features, put the in a list of 'cat_feat'
# - iterating through the list do crosstab, and apply chi-square, and find extreme results
#   (rejecting the null that the data are independent), and one where it is less extreme 

In [0]:
#divide dataframe into 2 by approved and rejected credit cards application results
df_pos = df_clean[df_clean.A16 == '+'].copy()
df_neg = df_clean[df_clean.A16 == '-'].copy()

#check if it's done correctly
print(df_pos.A16.unique())
print(df_neg.A16.unique())

In [61]:
df_pos.dtypes

A1      object
A2      object
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14     object
A15      int64
A16     object
dtype: object

In [0]:
#check the df dtypes 
df_pos.dtypes

#put continuous and categorical features into lists
# cont_feat_0 = ['A2','A3','A8','A11','A14','A15']
cont_feat = ['A3','A8','A11','A15']

cat_feat =  ['A1','A4','A5','A6','A7','A9','A10','A12','A13']

In [0]:
#t_test for two independent means in t-distribution
def t_test_two_ind_means(a, b, alpha=0.025, tails=2):
   
    #transform data series into arrays and calculate their lengths and variances
    a, b = np.array(a), np.array(b)
    na, nb = len(a), len(b)
    va, vb = a.var(), b.var()
    
    #degree of freedom, following the conservative way, choosing the minimum from two
    df = min(na - 1, nb - 1)
    
    #the difference btw two samples' means and SE for the two means
    mean_diff = a.mean() - b.mean()
    stn_error = np.sqrt(va/na + vb/nb)
    
    #t_stats and p_value
    t_stat = mean_diff / stn_error
    p_value = tails * stats.t.sf(abs(t_stat), df)
    
    #confidence levels
    ci_low = mean_diff - stats.t.ppf(1-alpha, df) * stn_error
    ci_high = mean_diff + stats.t.ppf(1-alpha, df) * stn_error
   
    return (t_stat, p_value, mean_diff, ci_low, ci_high)

In [94]:
# t_test for continuous features in 'cont_feat' list 
t_tests = {}

for i in range (0, len(cont_feat)):
  t_tests[cont_feat[i]] = t_test_two_ind_means(df_pos[cont_feat[i]], df_neg[cont_feat[i]])

approved = []
rejected = []
  
#print feature name, t-stat, and p-value for each feature

#if t_stat > 0, approved t-stat is higher, add to approved list and 
#if t_stat < 0, rejected t-stat is higher, add to rejected list

for feature in t_tests.keys():
  print (feature, 'T-Statistic:', t_tests[feature][0], 'p-value:', t_tests[feature][1])
  if t_tests[feature][0] > 0:
    approved.append(feature)
  elif t_tests[feature][0] < 0:  
    rejected.append(feature)
    
#select one feature that is extremely different between the classes, and another feature that is notably less different

A3 T-Statistic: 5.299324589493291 p-value: 2.2795264590997203e-07
A8 T-Statistic: 8.511096587423829 p-value: 8.860679585039269e-16
A11 T-Statistic: 10.507033700624996 p-value: 3.8154625022031593e-22
A15 T-Statistic: 4.0865957676287366 p-value: 5.6476391645654614e-05


In [92]:
print(approved)
print(rejected)

['A3', 'A8', 'A11', 'A15']
[]


In [0]:
# Visualize the resulting data in a box plot
def plot_data(dictionary):
  labels, data = [*zip(*dictionary.items())]  # 'transpose' dict items to parallel key, value lists
  plt.figure(figsize=(10,10))
  plt.boxplot(data, whis=0, meanline=True, )
  plt.xticks(range(1, len(labels) + 1), labels)
  plt.show()


In [0]:
plot_data(t_tests)
# from the plot it is seen that A15 issue had a significant effect on approval decision

In [0]:
#create crosstabs for categorical features, run chisquare test, plot histograms
#for each of them

#The null hypothesis is that the rows/cols are independent -> low chi square
#"less extreme" just means smaller test statistic/larger p-value

cat_feat =  ['A1','A4','A5','A6','A7','A9','A10','A12','A13']

chisquare_results = {}
 
def create_plot_crosstabs(df):
  for feature in cat_feat:
    df_cross = pd.crosstab(df.A16, df[feature], normalize=True)
    chisquare_results[feature] = chisquare(df_cross, axis=None)
    print(df_cross)
    print(chisquare(df_cross, axis=None))
    df_cross.plot.bar();

create_plot_crosstabs(df_clean)    

In [86]:
chisquare_results

{'A1': Power_divergenceResult(statistic=0.15467544071536954, pvalue=0.9845514506871024),
 'A10': Power_divergenceResult(statistic=0.23302275514822626, pvalue=0.9720900555984554),
 'A12': Power_divergenceResult(statistic=0.017567640457870255, pvalue=0.9993839708701547),
 'A13': Power_divergenceResult(statistic=1.5495944034952358, pvalue=0.9072766507666103),
 'A4': Power_divergenceResult(statistic=0.9415021727965406, pvalue=0.9671446983396204),
 'A5': Power_divergenceResult(statistic=0.9415021727965406, pvalue=0.9671446983396204),
 'A6': Power_divergenceResult(statistic=0.577330684858903, pvalue=1.0),
 'A7': Power_divergenceResult(statistic=2.7089601767317295, pvalue=0.9999668152444824),
 'A9': Power_divergenceResult(statistic=0.5426034628725004, pvalue=0.9094367325568282)}

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.



- I ran **t-test** to compare two independent means between approved and rejected credit cards applications data for continuous features, with significance level = 0.025. The positive T-statistic indicates the mean value of the variable is larger for approved applications then than for rejected.The difference between T-Statistic-values is the **most extreme** for **A11** feature with Tstatistics-value = 10.50 and extremely small p-value: 3.8154625022031593e-22. The feature that is **notably less different** is **A15**, with T-Statistic-value =  4.08, and p-value: 5.6476391645654614e-05, although it's still to low to reject the null hypothesis with significance level 0.025.



- The **chisquare test** yields high results for all categorical features with p-value > 0.05, which means  that the credit cards approval/rejection is highly dependent on all features, though the **most extreme** results are for  **A6** feature with p-value=1.0 and **A7** feature with p-value = 0.99. The **less extreme** result was for **A12** feature with statistic = 0.0175 and p-value = 0.9072.

- the most challenging for this sprint challenge was the interpretation of the t-test results and visualization, since we're limited in time. During the week, I tried to put the code for the homework assigments into reusable functions, which I used today and it helped me and saved my time a ton.
